In [8]:
import time

import cv2
import numpy as np

confid = 0.3
thresh = 0.3

#vid_path = r"C:\Users\MONSTER\Social-Distancing-Analyser-COVID-19-master\videos\UntitledProjectggggggggggg.mp4"
vid_path = r"C:\Users\MONSTER\Social-Distancing-Analyser-COVID-19-master\videos\taksim3.mp4"

# Calibration needed for each video
#Her video için gereken kalibrasyon ayarları yapılıyor.
def calibrated_dist(p1, p2):
    return ((p1[0] - p2[0]) ** 2 + 550 / ((p1[1] + p2[1]) / 2) * (p1[1] - p2[1]) ** 2) ** 0.5


def isclose(p1, p2):
    c_d = calibrated_dist(p1, p2)
    calib = (p1[1] + p2[1]) / 2
    if 0 < c_d < 0.15 * calib:
        return 1
    elif 0 < c_d < 0.2 * calib:
        return 2
    else:
        return 0

#YOLO modelinin eğittiği COCO sınıf etiketleri dosya yolundan yükleniyor
labelsPath = r"C:\Users\MONSTER\Untitled Folder 3\yolo-coco\coco.names"
LABELS = open(labelsPath).read().strip().split("\n")

np.random.seed(42)

#YOLO yolları (YOLO weight ve YOLO configuration) dosya yolundan yükleniyor
weightsPath = r"C:\Users\MONSTER\Social_Distancing_with_AI-master\Social_Distancing_with_AI-master\Models\yolov3.weights"
configPath = r"C:\Users\MONSTER\Social_Distancing_with_AI-master\Social_Distancing_with_AI-master\Models\yolov3.cfg"

###### use this for faster processing (caution: slighly lower accuracy) ###########

# weightsPath = "./yolov3-tiny.weights"  ## https://pjreddie.com/media/files/yolov3-tiny.weights
# configPath = "./yolov3-tiny.cfg"       ## https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-tiny.cfg

#YOLO yolları kullanılarak model belleğe yükleniyor

#COCO veri kümesinde eğitilmiş (80 sınıf) YOLO nesne dedektörü yükleniyor
#OpenCV'nin dnn modülü yardımıyla YoloV3 ağırlıklarını ve konfigürasyon dosyaları yükleniyor.
print("[INFO] YOLO diskten yükleniyor...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

#YOLO'dan yalnızca ihtiyacımız olan *output* katman adlarını belirleniyor
#YOLO'dan çıktı katmanlarının indislerini topluyoruz; Sonuçlarımızı işlemek için onlara ihtiyacımız olacak.
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

#video akışı ve işaretçi video dosyasının çıktısını almak için başlatılıyor
#video akışımızı başlatıyoruz.
print("[INFO] video akışına erişiyor...")
vs = cv2.VideoCapture(vid_path)
writer = None

(W, H) = (None, None)

fl = 0
q = 0

#Son olarak, çerçeveleri işlemeye ve insanların güvenli sosyal mesafeyi koruyup korumadığını belirlemeye hazırız:
#video akışından kareler üzerinde döngü kuruluyor.
while True:
    #dosyadan sonraki kareyi okuyarak, kareler üzerinde döngü başlar
    (grabbed, frame) = vs.read()
    
    #çerçeve alınmamışsa, akışın sonuna ulaştık
    if not grabbed:
        break

    if W is None or H is None:
        (H, W) = frame.shape[:2] #ölçekleme amacıyla çerçeve boyutlarını yakalar.
        q = W

    frame = frame[0:H, 200:q] #tahmin için çerçeve yeniden boyutlandırılıyor.
    (H, W) = frame.shape[:2] #ölçekleme amacıyla çerçeve boyutlarını yakalar.
    
    #Derin sinir ağlarına sahip nesneleri doğru şekilde tahmin etmek için, verilerimizi önceden işlememiz gerekir ve 
    #cv2.dnn modülü bize bu amaçla iki işlev sağlar: blobFromImage ve blobFromImages. 
    #Bu işlevler, isteğe bağlı olarak ölçekleme, ortalama çıkarma ve kanal değiştirme gerçekleştirir.  
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
                                 swapRB=True, crop=False) #nesneleri algılama. 
    # ScaleFactor ait 0.00392 olarak da yazılabilir 1/255 . Bu nedenle, görüntü piksellerini 0 ila 1 aralığına ölçeklendiriyoruz.
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    #listeler oluşturuluyor
    boxes = [] #sınırlayıcı kutular
    confidences = [] #nesne algılama güvenlerimizi
    classIDs = []

    for output in layerOutputs:

        for detection in output:

            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if LABELS[classID] == "person":

                if confidence > confid:
                    #sınırlayıcı kutunun koordinatları hesaplanıyor ve daha sonra merkez noktası türetiliyor, 
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")

                    #Sınırlayıcı kutunun üst ve sol köşesini türetmek için merkez (x, y) koordinatlarını kullanır.
                    #Sınırlayıcı kutu koordinatlarını kullanarak, daha sonra nesnenin sol üst koordinatlarını türetilir.
                    
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))

                    #listeler güncelleniyor
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)
                    
                    
#Zayıf, çakışan sınırlayıcı kutuları bastırmak için non-maxima bastırma uygulanıyor.
    #non-maxima bastırma: birçok örtüşen varlık arasından tek bir varlık seçen bir bilgisayarlı görme yöntemidir 
    #Maksimal olmayan bastırmanın amacı zayıf, çakışan sınırlayıcı kutuları bastırmaktır.
    #(OpenCV için yerleşiktir) ve algılamalar idx'leri ile sonuçlanır.
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confid, thresh)

    #non-maxima bastırmanın en az bir algılama sağladığı kontrol ediliyor
    if len(idxs) > 0:

        status = list()
        idf = idxs.flatten()
        close_pair = list()
        s_close_pair = list()
        center = list()
        dist = list()
        
        
        #tuttuğumuz dizinler üzerinde döngü başlatılıyor.
        for i in idf:
            #boundbox sınırlayıcı kutu koordinatları elde ediliyor.
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            #her kişi tespitinin güvenliği, sınırlayıcı kutusu ve merkezi güncelleniyor. kişinin merkezi elde ediliyor.

            center.append([int(x + w / 2), int(y + h / 2)])

            status.append(0)
        for i in range(len(center)):
            for j in range(len(center)):
                g = isclose(center[i], center[j])

                if g == 1:

                    close_pair.append([center[i], center[j]])
                    status[i] = 1
                    status[j] = 1
                elif g == 2:
                    s_close_pair.append([center[i], center[j]])
                    if status[i] != 1:
                        status[i] = 2
                    if status[j] != 1:
                        status[j] = 2
        #video üzerine yazdırılacak yazılar için değişkenler değerleri tutuyor

        total_p = len(center)
       # low_risk_p = status.count(2)
        high_risk_p = status.count(1)
        safe_p = status.count(0)
        kk = 0

        #frame üzerine sonuçlar yazdırılıyor
        for i in idf:

            #sub_img = frame[10:170, 10:W - 10]
           # black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0

           # res = cv2.addWeighted(sub_img, 0.77, black_rect, 0.23, 1.0)

            #frame[10:170, 10:W - 10] = res

           # cv2.putText(frame, "            Sosyal Mesafe Tespit", (210, 45),
            #            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            #cv2.rectangle(frame, (20, 60), (510, 160), (170, 170, 170), 2)
            #cv2.putText(frame, "Connecting lines shows closeness among people. ", (30, 80),
             #           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
           # cv2.putText(frame, "-- SARI    : YAKIN", (50, 110),
          #              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
          #  cv2.putText(frame, "-- KIRMIZI : COK YAKIN", (50, 130),
          #              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

           # cv2.rectangle(frame, (535, 60), (W - 20, 160), (170, 170, 170), 2)
            #cv2.putText(frame, "Bounding box shows the level of risk to the person.", (545, 80),
             #           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
         #   cv2.putText(frame, "-- KOYU KIRMIZI: YUKSEK RISK", (565, 110),
          #              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 150), 1)
          #  cv2.putText(frame, "-- TURUNCU    : DUSUK RISK", (565, 130),
          #              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 120, 255), 1)

           # cv2.putText(frame, "-- YESIL        : GUVENLI", (565, 150),
            #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

         #   tot_str =  "TOPLAM SAYI  : " + str(total_p)
         #   high_str = "YUKSEK RISK : " + str(high_risk_p)
          #  low_str =  "DUSUK RISK  : " + str(low_risk_p)
         #   safe_str = "GUVENLI      : " + str(safe_p)
            text = "Sosyal Mesafe ihlali: "+str(high_risk_p)
            cv2.putText(frame, text, (10, frame.shape[0] - 25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)
         #   sub_img = frame[H - 120:H, 0:210]
         #   black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0

         #   res = cv2.addWeighted(sub_img, 0.8, black_rect, 0.2, 1.0)

        #    frame[H - 120:H, 0:210] = res

         #   cv2.putText(frame, tot_str, (10, H - 90),
          #              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
         #   cv2.putText(frame, safe_str, (10, H - 65),
         #               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)
          #  cv2.putText(frame, low_str, (10, H - 40),
         #               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 120, 255), 1)
        #    cv2.putText(frame, high_str, (10, H - 15),
        #                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 150), 1)

            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            if status[kk] == 1:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 150), 2)

            elif status[kk] == 0:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

            #else:
            #    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 120, 255), 2)

            kk += 1
        for h in close_pair:
            cv2.line(frame, tuple(h[0]), tuple(h[1]), (0, 0, 255), 2)
        for b in s_close_pair:
       #     cv2.line(frame, tuple(b[0]), tuple(b[1]), (0, 255, 255), 2)
            cv2.line(frame, tuple(b[0]), tuple(b[1]), (0, 0, 255), 2)
        cv2.imshow('Social distancing analyser', frame)
        #cv2.waitKey(1)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        #frame mp4 video olarak dosyaya kaydediliyor
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MP4V")
        writer = cv2.VideoWriter(r"C:\Users\MONSTER\Social-Distancing-Analyser-COVID-19-master\sonuclar\taksim3.mp4", fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)

    writer.write(frame)
print("Processing finished: open taksim3.mp4")
writer.release()
vs.release()
cv2.destroyAllWindows()

[INFO] YOLO diskten yükleniyor...
[INFO] video akışına erişiyor...
Processing finished: open taksim3.mp4
